In [1]:
using JuMP,HiGHS,LaTeXStrings, GLPK, StatsPlots, Plots, DelimitedFiles

# Input Data

In [2]:
#Input data

asset_quantity = readdlm("data/sample_asset_quantity.csv", ',', Float64)
asset_tiers = readdlm("data/sample_asset_tiers.csv", ',', Float64)
asset_value = readdlm("data/sample_asset_value.csv", ',', Float64)

account_exposure = readdlm("data/sample_account_exposure.csv", ',', Float64)
account_durations = readdlm("data/sample_account_duration.csv", ',', Float64)


haircuts = readdlm("data/sample_haircuts.csv", ',', Float64)/100



10×5 Matrix{Float64}:
 0.9   0.909091  0.909091  0.943396  0.9
 1.0   1.0       0.909091  0.900901  1.0
 1.0   1.0       1.0       1.0       1.0
 1.0   0.909091  0.909091  0.900901  1.0
 0.9   0.934579  0.909091  0.943396  0.9
 0.9   0.909091  0.909091  0.943396  0.9
 0.95  1.0       0.980392  1.0       0.95
 0.95  1.0       0.980392  1.0       0.95
 0.95  1.0       1.0       1.0       0.95
 0.95  0.952381  0.980392  0.952381  0.95

In [3]:
# Auxillary variables

tier_set = [0.2, 0.5, 0.8]

n_assets = length(asset_tiers)

n_accounts = length(account_durations)



cost_factor_matrix = Array{Float64}(undef, length(asset_tiers), length(account_durations))

for i in 1:n_assets
    for j in 1:n_accounts
        cost_factor_matrix[i,j] = abs(asset_tiers[i] - account_durations[j])
    end
end

cost_factor_matrix = round.(cost_factor_matrix, digits = 2);

print(cost_factor_matrix)




[

0.2 0.2 0.8 0.8 0.8; 0.2 0.2 0.8 0.8 0.8; 0.2 0.2 0.8 0.8 0.8; 0.2 0.2 0.8 0.8 0.8; 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5; 0.8 0.8 0.2 0.2 0.2; 0.8 0.8 0.2 0.2 0.2; 0.8 0.8 0.2 0.2 0.2; 0.8 0.8 0.2 0.2 0.2

]

# Constructing JuMP model

In [4]:

model = Model()

#Q 
@variable(model, Q[i = 1:n_assets, j = 1:n_accounts], lower_bound = 0, upper_bound = 1)


#Consistency constraint
@constraint(model,[i = 1:n_assets],sum(Q[i,j] for j in 1:n_accounts) <= 1)

#Requirement constraint
@constraint(model, [j = 1:n_accounts],
    sum(Q[i,j]*asset_quantity[i]*asset_value[i]*haircuts[i,j] for i in 1:n_assets )
    >= account_exposure[j])



#Objective_function
@objective(model, Min, sum(Q[i,j]*cost_factor_matrix[i,j] for i in 1:n_assets,j in 1:n_accounts));
print(model)

Min 0.2 Q[1,1] + 0.2 Q[1,2] + 0.8 Q[1,3] + 0.8 Q[1,4] + 0.8 Q[1,5] + 0.2 Q[2,1] + 0.2 Q[2,2] + 0.8 Q[2,3] + 0.8 Q[2,4] + 0.8 Q[2,5] + 0.2 Q[3,1] + 0.2 Q[3,2] + 0.8 Q[3,3] + 0.8 Q[3,4] + 0.8 Q[3,5] + 0.2 Q[4,1] + 0.2 Q[4,2] + 0.8 Q[4,3] + 0.8 Q[4,4] + 0.8 Q[4,5] + 0.5 Q[5,1] + 0.5 Q[5,2] + 0.5 Q[5,3] + 0.5 Q[5,4] + 0.5 Q[5,5] + 0.5 Q[6,1] + 0.5 Q[6,2] + 0.5 Q[6,3] + 0.5 Q[6,4] + 0.5 Q[6,5] + 0.8 Q[7,1] + 0.8 Q[7,2] + 0.2 Q[7,3] + 0.2 Q[7,4] + 0.2 Q[7,5] + 0.8 Q[8,1] + 0.8 Q[8,2] + 0.2 Q[8,3] + 0.2 Q[8,4] + 0.2 Q[8,5] + 0.8 Q[9,1] + 0.8 Q[9,2] + 0.2 Q[9,3] + 0.2 Q[9,4] + 0.2 Q[9,5] + 0.8 Q[10,1] + 0.8 Q[10,2] + 0.2 Q[10,3] + 0.2 Q[10,4] + 0.2 Q[10,5]
Subject to


 337580.7187721691 Q[1,1] + 1.416835560987015e6 Q[2,1] + 242181.403673832 Q[3,1] + 530751.1039390559 Q[4,1] + 1.9252052101651952e6 Q[5,1] + 2.48391407769804e6 Q[6,1] + 98653.12350377608 Q[7,1] + 990856.6525174977 Q[8,1] + 146168.41998874999 Q[9,1] + 90366.7778 Q[10,1] >= 824000.0
 340990.62502580293 Q[1,2] + 1.416835560987015e6 Q[2,2] + 242181.403673832 Q[3,2] + 482501.0035857849 Q[4,2] + 1.9991746731706962e6 Q[5,2] + 2.509004118911999e6 Q[6,2] + 103845.39316186956 Q[7,2] + 1.0430070026499976e6 Q[8,2] + 153861.494725 Q[9,2] + 90593.26095419283 Q[10,2] >= 669292.0
 340990.62502580293 Q[1,3] + 1.288032328182894e6 Q[2,3] + 242181.403673832 Q[3,3] + 482501.0035857849 Q[4,3] + 1.9446517274590374e6 Q[5,3] + 2.509004118911999e6 Q[6,3] + 101809.20898609381 Q[7,3] + 1.0225558849898352e6 Q[8,3] + 153861.494725 Q[9,3] + 93257.76863099477 Q[10,3] >= 548600.0
 353858.19577229326 Q[1,4] + 1.2764284333203589e6 Q[2,4] + 242181.403673832 Q[3,4] + 478154.14769236505 Q[4,4] + 2.0180348114616266e6 Q[5,4] 


 Q[7,2] >= 0.0
 Q[8,2] >= 0.0
 Q[9,2] >= 0.0
 Q[10,2] >= 0.0
 Q[1,3] >= 0.0
 Q[2,3] >= 0.0
 Q[3,3] >= 0.0
 Q[4,3] >= 0.0
 Q[5,3] >= 0.0
 Q[6,3] >= 0.0


 Q[7,3] >= 0.0
 Q[8,3] >= 0.0
 Q[9,3] >= 0.0
 Q[10,3] >= 0.0
 Q[1,4] >= 0.0
 Q[2,4] >= 0.0
 Q[3,4] >= 0.0
 Q[4,4] >= 0.0
 Q[5,4] >= 0.0
 Q[6,4] >= 0.0
 Q[7,4] >= 0.0
 Q[8,4] >= 0.0
 Q[9,4] >= 0.0
 Q[10,4] >= 0.0
 Q[1,5] >= 0.0
 Q[2,5] >= 0.0
 Q[3,5] >= 0.0
 Q[4,5] >= 0.0
 Q[5,5] >= 0.0
 Q[6,5] >= 0.0
 Q[7,5] >= 0.0
 Q[8,5] >= 0.0
 Q[9,5] >= 0.0
 Q[10,5] >= 0.0
 Q[1,1] <= 1.0
 Q[2,1] <= 1.0
 Q[3,1] <= 1.0
 Q[4,1] <= 1.0
 Q[5,1] <= 1.0
 Q[6,1] <= 1.0
 Q[7,1] <= 1.0
 Q[8,1] <= 1.0
 Q[9,1] <= 1.0
 Q[10,1] <= 1.0
 Q[1,2] <= 1.0
 Q[2,2] <= 1.0
 Q[3,2] <= 1.0
 Q[4,2] <= 1.0
 Q[5,2] <= 1.0
 Q[6,2] <= 1.0
 Q[7,2] <= 1.0
 Q[8,2] <= 1.0
 Q[9,2] <= 1.0
 Q[10,2] <= 1.0
 Q[1,3] <= 1.0
 Q[2,3] <= 1.0
 Q[3,3] <= 1.0
 Q[4,3] <= 1.0
 Q[5,3] <= 1.0
 Q[6,3] <= 1.0
 Q[7,3] <= 1.0
 Q[8,3] <= 1.0
 Q[9,3] <= 1.0
 Q[10,3] <= 1.0
 Q[1,4] <= 1.0
 Q[2,4] <= 1.0
 Q[3,4] <= 1.0
 Q[4,4] <= 1.0
 Q[5,4] <= 1.0
 Q[6,4] <= 1.0
 Q[7,4] <= 1.0
 Q[8,4] <= 1.0
 Q[9,4] <= 1.0
 Q[10,4] <= 1.0
 Q[1,5] <= 1.0
 Q[2,5] <= 1.0
 Q[

Q[9,5] <= 1.0
 Q[10,5] <= 1.0


# Solving with HiGHs


In [5]:
set_optimizer(model, HiGHS.Optimizer)
set_optimizer_attribute(model, "solver", "simplex")
optimize!(model)

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
15 rows, 50 cols, 100 nonzeros
15 rows, 50 cols, 100 nonzeros
Presolve : Reductions: rows 15(-0); columns 50(-0); elements 100(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 5(5036.59) 0s
          6     4.2998165714e-01 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 6
Objective value     :  4.2998165714e-01
HiGHS run time      :          0.03


In [6]:

println("Objective value is $(objective_value(model))")


Objective value is 0.4299816571359823


# Export to .csv

In [7]:
Q_values_HiGHS_simplex = zeros((n_assets, n_accounts))


for i in 1:n_assets
    for j in 1:n_accounts
        Q_values_HiGHS_simplex[i,j] = value(Q[i,j])
        if value(Q[i,j]) != 0 
            println("Q_{$(i), $(j)} = $(value(Q[i,j]))")
        end
    end
end

println(Q_values_HiGHS_simplex)
#store all Values to an array


# Uncomment line below to save to csv files

#writedlm("Q_values_HiGHS_simplex_no_single_limits.csv", Q_values_HiGHS_simplex, ',')

Q_{2, 1} = 0.5815777234063592
Q_{2, 2} = 0.41842227659364073
Q_{6, 2} = 0.030472823235595
Q_{6, 5} = 0.19002267600070313
Q_{8, 3} = 0.5364987948853802
Q_{8, 4} = 0.062170742703785954


[0.0 0.0 0.0 0.0 0.0; 0.5815777234063592 0.41842227659364073 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.030472823235595 0.0 0.0 0.19002267600070313; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.5364987948853802 0.062170742703785954 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
